In [14]:
import urllib.request
import json
import pandas as pd
import requests
import shutil
import os


url = (
    "https://pixabay.com/api/?key=42732485-b4c1847a3529ea910e8e5fdeb&q=animaux&image_type=photo&per_page=100"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))
responsedata

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_link = os.path.basename(url)
        filename = os.path.join("./images/", image_link)
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []

for result in responsedata["hits"]:
    array.append(
        (
            result["id"],
            result["tags"],
            result["largeImageURL"],
            result["imageWidth"],
            result["imageHeight"],
            result["imageSize"],
            result["likes"], 
            result["user"],
            result["largeImageURL"].split('.')[-1]
        )
    )


dataframe = pd.DataFrame(array, columns=["id", "tags", "largeImageURL", "imageWidth","imageHeight","imageSize", "likes", "user", "format"])
dataframe = dataframe.astype(
    dtype={"id": "<U200", "tags": "<U200", "largeImageURL": "<U200","imageWidth": "<U200","imageHeight": "<U200","imageSize": "<U200", "likes": "<U200", "user": "<U200", "format": "<U200"}
)
# Data type conversion to sort
for int_column in ["id","imageWidth","imageHeight","imageSize","likes"]:
    dataframe[int_column] = dataframe[int_column].astype(int)

#dataframe.largeImageURL.apply(download_image)
#dataframe

# Sort
sort_key = input("Sort by : \n id / imageWidth / imageHeight / imageSize / color / likes ")
sorted_dataframe = dataframe.sort_values(by=sort_key)
# Json conversion
json_data = sorted_dataframe.to_json(orient='records', indent = 4)
file_path = "data.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)
# Storage

# Read test for the json produced
readtest_data = json.load(open("data.json"))
readtest_dataframe = pd.json_normalize(readtest_data)
readtest_dataframe
 

,id,tags,largeImageURL,imageWidth,imageHeight,imageSize,likes,user,format
0,17344,"animals, lions, safari",https://pixabay.com/get/gc5cdde21c772cb773b489...,3083,2055,1697541,424,PublicDomainPictures,jpg
1,19830,"flowers, butterfly, silver washed fritillary",https://pixabay.com/get/g0a3596285291eb68b7462...,2144,1424,668020,5427,Larisa-K,jpg
2,21787,"lions, animals, zoo",https://pixabay.com/get/g3662c80aa938439e01c7e...,5000,3333,3493038,534,PublicDomainPictures,jpg
3,50267,"owl, bird, animal",https://pixabay.com/get/gf5d957c7e014d11277c6e...,2122,1593,957134,3277,Chraecker,jpg
4,61189,"koalas, animals, wildlife",https://pixabay.com/get/g24ab1697cf7c794c81670...,4374,2844,3831312,479,Holgi,jpg
...,...,...,...,...,...,...,...,...,...
95,4551002,"birds, asian green bee-eater, ornithology",https://pixabay.com/get/gc53b18bbe1da1b62001c1...,3000,2000,698745,513,Nghangvu,jpg
96,4649468,"animals, icelandic horses, mammal",https://pixabay.com/get/g7249c884b3c077bd02e44...,6970,4356,6509975,440,susnpics,jpg
97,4849979,"chicken, rooster, hen",https://pixabay.com/get/gae4dc7946351a75ecf7b8...,4000,2666,2448284,554,Danganhfoto,jpg
98,7576994,"bird, white-cheeked turaco, tauraco leucotis",https://pixabay.com/get/g0d25f6f8b9e50f886656a...,6000,4000,4956004,484,Marjonhorn,jpg


On va stocker id, tags, largeImageURL, imageWidth, imageHeight, imageSize, likes, format de l'image en prenant ce qu'il y a après le .; user

In [23]:
#Test code
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans

imgfile = Image.open("loup.jpg")
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = KMeans(n_clusters=4, n_init=2)
clusters.fit(numarray)
npbins = numpy.arange(0, 5)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
barlist = plot.bar(labels, histogram[0])
for i in range(4):
    barlist[i].set_color(
        "#%02x%02x%02x"
        % (
            math.ceil(clusters.cluster_centers_[i][0]),
            math.ceil(clusters.cluster_centers_[i][1]),
            math.ceil(clusters.cluster_centers_[i][2]),
        )
    )
plot.show()

KeyboardInterrupt: 